In [1]:
import requests
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment, PatternFill, Border, Side


In [2]:
df_oeufs_fr = pd.read_csv("../data/dfoeufs_fr.csv")
df_oeufs_fr

,code,categories_tags,labels_tags,product_name,generic_name,quantity,product_quantity_unit,product_quantity,allergens_tags,ingredients_tags,ingredients,countries_tags,w_eggs,breeding,has_breeding_type,has_egg_weight,has_egg_weight_s,french,french_s
0,00003100,['en:farming-products' 'en:eggs'],[],"[{'lang': 'main', 'text': 'Hard Boiled Eggs'}\...",[],2,NaN,0.0,['en:eggs'],['fr:eggs' 'en:e330' 'fr:sodium-benzoate' 'fr:...,"[{""percent_max"":100.0,""percent_min"":100.0,""is_...",['en:france'],100.0,NaN,Aucun,True,has weight,True,français
1,00257961,['en:farming-products' 'en:eggs'],NaN,"[{'lang': 'main', 'text': '6 Oeufs Bleus'}\n {...",[],NaN,NaN,NaN,[],NaN,NaN,['en:france' 'en:united-kingdom'],0.0,NaN,Aucun,False,no weight,True,français
2,0055000002240,['en:farming-products' 'en:eggs'],[],"[{'lang': 'main', 'text': 'Œufs'} {'lang': 'fr...",[],NaN,NaN,NaN,[],NaN,NaN,['en:france'],0.0,NaN,Aucun,False,no weight,True,français
3,0061719011930,['en:farming-products' 'en:eggs' 'en:chicken-e...,[],"[{'lang': 'main', 'text': '12 œufs blanc calib...",[],12,NaN,0.0,[],NaN,NaN,['en:france'],600.0,barn,computed,True,has weight,True,français
4,00901796,['en:farming-products' 'en:eggs' 'en:chicken-e...,['en:green-dot' 'fr:categorie-a'],"[{'lang': 'main', 'text': '6 Œufs Moyens de Po...",[],6 œufs,NaN,0.0,['en:eggs'],['en:egg'],"[{""percent_max"":100.0,""percent_min"":100.0,""is_...",['en:france' 'en:united-kingdom'],300.0,free_range,free_range,True,has weight,True,français
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2612,0764450111661,['en:farming-products' 'en:eggs'],['en:nutriscore' 'fr:bleu-blanc-coeur'],[],[],NaN,NaN,NaN,[],NaN,NaN,['en:france'],0.0,NaN,Aucun,False,no weight,True,français
2613,3700864040064,['en:farming-products' 'en:eggs' 'en:chicken-e...,['en:nutriscore'],"[{'lang': 'main', 'text': 'Gros oeufs plein ai...",[],NaN,NaN,NaN,[],NaN,NaN,['en:france'],0.0,free_range,free_range,False,no weight,True,français
2614,3700133918872,['en:farming-products' 'en:eggs' 'en:chicken-e...,['fr:label-rouge'],"[{'lang': 'main', 'text': 'Oeufs frais plein a...",[],NaN,NaN,NaN,[],NaN,NaN,['en:france'],0.0,free_range,free_range,False,no weight,True,français
2615,3700864017127,['en:farming-products' 'en:eggs' 'en:chicken-e...,[],"[{'lang': 'main', 'text': 'Gros oeufs plein ai...",[],NaN,NaN,NaN,[],NaN,NaN,['en:france'],0.0,free_range,free_range,False,no weight,True,français


In [61]:
dfoeufs_fr_no_breeding_no_quantity = df_oeufs_fr.loc[(df_oeufs_fr["has_breeding_type"] == "Aucun") & (df_oeufs_fr["has_egg_weight"] == False), 'code']
dfoeufs_fr_no_breeding = df_oeufs_fr.loc[(df_oeufs_fr["has_breeding_type"] == "Aucun") & (df_oeufs_fr["has_egg_weight"] == True), 'code']
dfoeufs_fr_no_quantity_not_free_range = df_oeufs_fr.loc[(df_oeufs_fr["has_breeding_type"] != "free_range") &(df_oeufs_fr["has_breeding_type"] != "Aucun") & (df_oeufs_fr["has_egg_weight"] == False), 'code']
dfoeufs_fr_no_quantity = df_oeufs_fr.loc[(df_oeufs_fr["has_breeding_type"] != "Aucun") & (df_oeufs_fr["has_egg_weight"] == False), 'code']
no_breeding_no_quantity = dfoeufs_fr_no_breeding_no_quantity.tolist()
no_breeding = dfoeufs_fr_no_breeding.tolist()
no_quantity = dfoeufs_fr_no_quantity.tolist()
no_quantity_not_free_range = dfoeufs_fr_no_quantity_not_free_range.tolist()


In [ ]:
# Importation des bibliothèques nécessaires
import os
import requests
import pandas as pd
from openpyxl import load_workbook, Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font

# Récupère les données d'un produit à partir de son code-barres via l'API Open Food Facts
def fetch_product_data(product_code: str) -> dict:
    url = (
        f"https://world.openfoodfacts.net/api/v2/product/{product_code}?fields="
        "code,image_url,images,categories_tags,product_quantity,product_quantity_unit,"
        "image_front_url,ingredients_tags,labels_tags,product_name,generic_name"
    )
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json().get("product", {})
    except requests.RequestException:
        print(f"Error fetching data for {product_code}")
        return {}

# Construit une ligne de données pour un produit donné
def build_row(product: dict, code: str, show_breeding: bool, show_quantity: bool) -> dict:
    # Extraction des champs utiles du JSON
    product_name = product.get("product_name", "")
    generic_name = product.get("generic_name", "")
    ingredients = product.get("ingredients_tags", [])
    labels = product.get("labels_tags", [])
    categories = product.get("categories_tags", [])

    front_url = product.get("image_front_url", "")
    image_url_base = product.get("image_url", "").rsplit("/", 1)[0] if product.get("image_url") else ""
    images_dict = product.get("images", {})
    image_names = [k for k in images_dict if k.isdigit()]

    # Liste jusqu'à 10 images différentes de l'image principale
    image_urls = [
        f"{image_url_base}/{name}.jpg"
        for name in image_names if f"{image_url_base}/{name}.jpg" != front_url
    ][:10]

    # Construction du dictionnaire représentant une ligne dans le tableau
    row = {
        "Traité": "",
        "Info invisible": "",
        "Pas œuf": "",
        "Image front": f'=IMAGE("{front_url}")' if front_url else "",
        "Code et URL produit": code,
        "Autres": f"{product_name}\n{generic_name}\n{ingredients}\n{labels}\n{categories}",
        **{f"Image {i+1}": f'=IMAGE("{image_urls[i]}")' if i < len(image_urls) else "" for i in range(10)}
    }

    # Ajout conditionnel des colonnes "Elevage", "Qté" et "Calibre"
    if show_breeding:
        row["Elevage"] = ""
    if show_quantity:
        row["Qté"] = ""
        row["Calibre"] = ""

    return row

# Fonction principale pour générer un fichier Excel à partir d'une liste de codes produits
def generate_product_excel(product_codes: list[str], output_excel_filename: str,
                           show_col_breeding: bool = True, show_cols_quantity: bool = True,
                           data_path: str = "../data/",
                           sheet_name: str = "Feuille1"):

    # S'assure que le nom de fichier a bien l'extension .xlsx
    if not output_excel_filename.endswith(".xlsx"):
        output_excel_filename += ".xlsx"

    file_path = data_path + output_excel_filename

    rows = []
    codes = []

    # Récupère les données de chaque produit et construit les lignes
    for code in product_codes:
        product = fetch_product_data(code)
        if product:
            rows.append(build_row(product, code, show_col_breeding, show_cols_quantity))
            codes.append(code)

    # Ordre et présence des colonnes selon les options
    column_order = ["Traité"]
    if show_col_breeding:
        column_order.append("Elevage")
    if show_cols_quantity:
        column_order.extend(["Qté", "Calibre"])
    column_order += [
        "Info invisible", "Pas œuf", "Image front", "Code et URL produit", "Autres"
    ] + [f"Image {i+1}" for i in range(10)]

    # Renommage des colonnes pour affichage dans Excel
    col_rename = {
        "Traité": "Traité",
        "Info invisible": "Info invisible",
        "Pas œuf": "Pas œuf / comment.",
        "Code et URL produit": "Code et URL produit",
        "Image front": "Image principale",
        "Autres": "Nom, ingrédients, labels, catégories",
    }
    if show_col_breeding:
        col_rename["Elevage"] = "Elevage\n (cage, sol, plein-air, bio ou 3, 2, 1, 0)"
    if show_cols_quantity:
        col_rename["Qté"] = "Quantité\n(nb d'œufs ou poids)"
        col_rename["Calibre"] = "Calibre\n(S, M, L, XL ou petit, moyen, gros, très gros)"

    # Création du DataFrame pandas
    df = pd.DataFrame(rows)
    df = df.reindex(columns=column_order)
    df.rename(columns=col_rename, inplace=True)

    # Création ou modification du fichier Excel
    if os.path.exists(file_path):
        wb = load_workbook(file_path)
        if sheet_name in wb.sheetnames:
            del wb[sheet_name]  # Supprime l'onglet s'il existe déjà
        ws = wb.create_sheet(title=sheet_name)
    else:
        wb = Workbook()
        ws = wb.active
        ws.title = sheet_name

    # Écriture des en-têtes
    for col_idx, col_name in enumerate(df.columns, start=1):
        ws.cell(row=1, column=col_idx, value=col_name)

    # Écriture des données
    for row_idx, row_data in enumerate(df.itertuples(index=False), start=2):
        for col_idx, cell_value in enumerate(row_data, start=1):
            ws.cell(row=row_idx, column=col_idx, value=cell_value)

    wb.save(file_path)

    # Mise en forme du fichier Excel
    format_excel_file(file_path, df.columns, codes, sheet_name=sheet_name)

# Mise en forme du fichier Excel généré
def format_excel_file(filename: str, columns: list[str], product_codes: list[str], sheet_name: str = "Feuille1"):
    wb = load_workbook(filename)
    ws = wb[sheet_name]

    # Ajout d’un lien hypertexte vers la fiche produit
    code_col_idx = list(columns).index("Code et URL produit") + 1
    for i, code in enumerate(product_codes):
        cell = ws.cell(row=i + 2, column=code_col_idx)
        cell.hyperlink = f"https://fr.openfoodfacts.org/produit/{code}"
        cell.style = "Hyperlink"

    # Définition des largeurs de colonnes
    widths = {
        "Traité": 8,
        "Info invisible": 10,
        "Pas œuf / comment.": 10,
        "Code et URL produit": 15,
        "Image principale": 50,
        "Nom, ingrédients, labels, catégories": 30,
        "Elevage\n (cage, sol, plein-air, bio ou 3, 2, 1, 0)": 15,
        "Quantité\n(nb d'œufs ou poids)": 13,
        "Calibre\n(S, M, L, XL ou petit, moyen, gros, très gros)": 17,
        **{f"Image {i}": 50 for i in range(1, 11)}
    }

    # Application des largeurs
    for idx, name in enumerate(columns, 1):
        ws.column_dimensions[get_column_letter(idx)].width = widths.get(name, 15)

    # Hauteur des lignes (pour les images)
    for r in range(2, 2 + len(product_codes)):
        ws.row_dimensions[r].height = 300

    # Style : fond gris et bordures légères
    fill_gray = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
    border = Border(
        left=Side(style="dotted", color="E0E0E0"),
        right=Side(style="dotted", color="E0E0E0"),
        top=Side(style="thin", color="AAAAAA"),
        bottom=Side(style="thin", color="AAAAAA")
    )

    # Alignement et bordures de toutes les cellules
    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
        for cell in row:
            cell.alignment = Alignment(wrap_text=True, vertical="center", horizontal="center")
            cell.border = border

    # Mise en forme de la première ligne (en-têtes)
    for cell in ws[1]:
        cell.alignment = Alignment(horizontal="center", wrap_text=True, vertical="center")
        cell.font = Font(size=8, bold=True)

    # Gel de la première ligne + colonnes avant l’image principale
    if "Image principale" in columns:
        freeze_col_idx = list(columns).index("Image principale") + 1
        ws.freeze_panes = ws.cell(row=2, column=freeze_col_idx)

        # Remplissage gris des colonnes gelées
        for r in range(2, ws.max_row + 1):
            for c in range(freeze_col_idx, ws.max_column + 1):
                ws.cell(row=r, column=c).fill = fill_gray

    wb.save(filename)
    print(f"✅ Fichier Excel généré : {filename}")


In [ ]:

'''# Test
sample_codes = ["00257961", "00388500", "0055000002240"]
generate_product_excel(
    product_codes=sample_codes,
    output_excel_filename="products_test.xlsx",
    show_col_breeding=True,
    show_cols_quantity=True,
    sheet_name="Test produits"
)
'''

In [70]:
generate_product_excel(
    product_codes=no_breeding_no_quantity,
    output_excel_filename="products_to_tag_27_05_25.xlsx",
    show_col_breeding=True,
    show_cols_quantity=True,
    sheet_name="no_breeding_no_quantity"
)


generate_product_excel(
    product_codes=no_breeding,
    output_excel_filename="products_to_tag_27_05_25.xlsx",
    show_col_breeding=True,
    show_cols_quantity=False,
    sheet_name="no_breeding"
)

generate_product_excel(
    product_codes=no_quantity_not_free_range,
    output_excel_filename="products_to_tag_27_05_25.xlsx",
    show_col_breeding=False,
    show_cols_quantity=True,
    sheet_name="no_quantity_not_free_range"
)

Error fetching data for 8794150111861
Error fetching data for 0764450111661
Error fetching data for 3760028590675
✅ Fichier Excel généré : ../data/products_to_tag_27_05_25.xlsx
Error fetching data for 3700036908284
✅ Fichier Excel généré : ../data/products_to_tag_27_05_25.xlsx
✅ Fichier Excel généré : ../data/products_to_tag_27_05_25.xlsx
